In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import umap
import joblib
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import KernelPCA, PCA
from sklearn.metrics import classification_report

In [113]:
test = joblib.load('./mpdocvectorsdict100.x')

In [115]:
x = np.array(test)

In [129]:
test_y = joblib.load('./all_vote_universal_credit_labels.y')

In [130]:
y = list(test_y.values())

In [63]:
one_count = 0
zero_count = 0
for i in y:
    if i == 0:
        zero_count += 1
    else:
        one_count += 1
print(zero_count)
print(one_count)

303
267


In [132]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
LogisticRegression(C=1)
scores = cross_val_score(reg, x, y, cv=10)
print(scores)
scores.mean()


[0.5        0.43103448 0.48275862 0.42105263 0.38596491 0.40350877
 0.50877193 0.41071429 0.51785714 0.58928571]


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

0.46509484919194544

In [2]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from collections import Counter
from scipy import sparse
import collections
import re
import joblib

# Custom Transformer that extracts columns passed as argument to its constructor
class TfidfFeatureExtractor(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self):
        self._feature_names = None
        self._useful_hashtags = None

    def vectoriser(self, features):
        dict_vectorizer = DictVectorizer()
        hashtags = list(features.values())
        sparse_adjacency_matrix = dict_vectorizer.fit_transform(hashtags)
        hashtags_df = pd.DataFrame(sparse_adjacency_matrix.todense(), index=features.keys()
                                   , columns=dict_vectorizer.get_feature_names())
        sparse_csr_matrix = sparse.csr_matrix(hashtags_df.values)
        return sparse_csr_matrix
    
    def features_transformer(self, X):
        new_features = collections.OrderedDict()
        features = X
        for mp_handle, feature_dict in features.items():
            new_feature_dict = {}
            for feature in self._useful_hashtags:
                if feature in feature_dict:
                    new_feature_dict[feature] = feature_dict[feature]
                else:
                    new_feature_dict[feature] = 0
            new_features[mp_handle] = new_feature_dict
        return new_features

    def hashtag_deleter(self, X):
        new_features = collections.OrderedDict()
        features = X
        for mp_handle, feature_dict in features.items():
            new_feature_dict = {}
            for feature in feature_dict:
                if feature in self._useful_hashtags:
                    new_feature_dict[feature] = feature_dict[feature]
            new_features[mp_handle] = new_feature_dict
        return new_features

    def mp_deleter(self, X, y):
        features = X
        handles = list(features.keys())
        for mp_handle in handles:
            if mp_handle not in y.keys():
                del features[mp_handle]
        return features

    def sanitise(self, mps_tweets_df):
        # Sanitise author column
        mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.lower())
        mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.split()[0])
        author_contents_mps_tweets_df = mps_tweets_df[['author', 'contents']].copy()

        def extract_hash_tags(s):
            return set([re.sub(r"#+", "#", k) for k in set(
                [re.sub(r"(\W+)$", "", j, flags=re.UNICODE) for j in
                 set([i for i in s.split() if i.startswith("#")])])])

        dict_of_mps = {k: v for k, v in author_contents_mps_tweets_df.groupby('author')['contents']}
        new_dict_of_mps = {}
        for k, v in dict_of_mps.items():
            vals = []
            for val in v:
                vals.append(extract_hash_tags(val))
            new_dict_of_mps[k] = vals

        for k in new_dict_of_mps.keys():
            new_dict_of_mps[k] = [item for sublist in new_dict_of_mps[k] for item in sublist]

        return new_dict_of_mps

    def extract_features(self, all_tweets):
        all_hashtags = []
        count = 0
        for dict_of_tweets in all_tweets[:-1]:
            all_hashtags.append(self.sanitise(pd.DataFrame.from_dict(dict_of_tweets['posts'])))
        final_hashtags_dict = {}
        for dict_of_hashtags in all_hashtags:
            for k, v in dict_of_hashtags.items():
                if k in final_hashtags_dict.keys():
                    val = final_hashtags_dict[k]
                    if val != None:
                        val = val + v
                    else:
                        val = v
                    final_hashtags_dict[k] = val
                else:
                    final_hashtags_dict[k] = v
        for k in final_hashtags_dict.keys():
            final_hashtags_dict[k] = dict(Counter(final_hashtags_dict[k]))
        return final_hashtags_dict

    def extract_feature_names(self, X):
        used_hashtags = []
        for hashtag_list in X.values():
            used_hashtags += list(hashtag_list.keys())
        return list(set(used_hashtags))

    def extract_mp_handles(self, X):
        return list(X.keys())

    def extract_useful_columns(self, X):
        useful_columns = []
        for i in range(0, X.shape[1]):
            result = np.where(X[:, i] > 0)
            if 5 < len(result[0]) < 500:
                useful_columns.append(i)
        return useful_columns
    
    def transform_y(self, X, y):
        handles = list(y.keys())
        for key in handles:
            if key not in X.keys():
                del y[key]
        return y

    # Return self nothing else to do here
    def fit(self, X, y=None):
        features = self.extract_features(X)
        self._feature_names = self.extract_feature_names(features)
#         features = self.mp_deleter(features, y)
        feature_vector = self.vectoriser(features)
        useful_columns = self.extract_useful_columns(feature_vector.toarray())
        self._useful_hashtags = [self._feature_names[i] for i in useful_columns]
        return self

        # Method that describes what we need this transformer to do

    def transform(self, X, y=None):
        features = self.extract_features(X)
        if y is not None:
            features = self.mp_deleter(features, y)
        features = self.features_transformer(features)
        if y is not None:
            y = self.transform_y(features, y)
        feature_vector = self.vectoriser(features)
#         return feature_vector, y
        return TfidfTransformer().fit_transform(feature_vector).toarray(), y


In [5]:
x = joblib.load('./data/mps_2017_tweets.pckle')
y = joblib.load('./data/2017_health_vote_labels.y')
real_y = list(y.values())
future_x = joblib.load('./data/mps_2019_tweets.pckle')
future_y = joblib.load('./data/2019_health_vote_labels.y')
real_future_y = list(future_y.values())

In [6]:
transformer = TfidfFeatureExtractor()
transformer = transformer.fit(x, y)
x, real_y = transformer.transform(x, y)
future_x, future_y = transformer.transform(future_x, future_y)

In [ ]:
reg = LogisticRegression(C=1)
reg.fit(x, real_y)
predictions = reg.predict(x)
print(classification_report(np.array(list(future_y.values())), predictions, labels=[0,1]))

In [7]:
# Custom Transformer that extracts columns passed as argument to its constructor
class TimeFeatureExtractor(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self):
        self._feature_names = None
        self._useful_hashtags = None

    def vectoriser(self, features):
        dict_vectorizer = DictVectorizer()
        hashtags = list(features.values())
        sparse_adjacency_matrix = dict_vectorizer.fit_transform(hashtags)
        hashtags_df = pd.DataFrame(sparse_adjacency_matrix.todense(), index=features.keys()
                                   , columns=dict_vectorizer.get_feature_names())
        sparse_csr_matrix = sparse.csr_matrix(hashtags_df.values)
        return sparse_csr_matrix
    
    def features_transformer(self, X):
        new_features = collections.OrderedDict()
        features = X
        for mp_handle, feature_dict in features.items():
            new_feature_dict = {}
            for feature in self._useful_hashtags:
                if feature in feature_dict:
                    new_feature_dict[feature] = feature_dict[feature]
                else:
                    new_feature_dict[feature] = 0
            new_features[mp_handle] = new_feature_dict
        return new_features

    def hashtag_deleter(self, X):
        new_features = collections.OrderedDict()
        features = X
        for mp_handle, feature_dict in features.items():
            new_feature_dict = {}
            for feature in feature_dict:
                if feature in self._useful_hashtags:
                    new_feature_dict[feature] = feature_dict[feature]
            new_features[mp_handle] = new_feature_dict
        return new_features

    def mp_deleter(self, X, y):
        features = X
        handles = list(features.keys())
        for mp_handle in handles:
            if mp_handle not in y.keys():
                del features[mp_handle]
        return features

    def sanitise(self, mps_tweets_df):
        # Sanitise author column
        mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.lower())
        mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.split()[0])
        author_contents_mps_tweets_df = mps_tweets_df[['author', 'contents']].copy()

        def extract_hash_tags(s):
            return set([re.sub(r"#+", "#", k) for k in set(
                [re.sub(r"(\W+)$", "", j, flags=re.UNICODE) for j in
                 set([i for i in s.split() if i.startswith("#")])])])

        dict_of_mps = {k: v for k, v in author_contents_mps_tweets_df.groupby('author')['contents']}
        new_dict_of_mps = {}
        for k, v in dict_of_mps.items():
            vals = []
            for val in v:
                vals.append(extract_hash_tags(val))
            new_dict_of_mps[k] = vals

        for k in new_dict_of_mps.keys():
            new_dict_of_mps[k] = [item for sublist in new_dict_of_mps[k] for item in sublist]

        return new_dict_of_mps

    def extract_features(self, all_tweets):
        all_hashtags = []
        count = 0
        for dict_of_tweets in all_tweets[:-1]:
            all_hashtags.append(self.sanitise(pd.DataFrame.from_dict(dict_of_tweets['posts'])))
        final_hashtags_dict = {}
        for dict_of_hashtags in all_hashtags:
            for k, v in dict_of_hashtags.items():
                if k in final_hashtags_dict.keys():
                    val = final_hashtags_dict[k]
                    if val != None:
                        val = val + v
                    else:
                        val = v
                    final_hashtags_dict[k] = val
                else:
                    final_hashtags_dict[k] = v
        for k in final_hashtags_dict.keys():
            final_hashtags_dict[k] = dict(Counter(final_hashtags_dict[k]))
        return final_hashtags_dict

    def extract_feature_names(self, X):
        used_hashtags = []
        for hashtag_list in X.values():
            used_hashtags += list(hashtag_list.keys())
        return list(set(used_hashtags))

    def extract_mp_handles(self, X):
        return list(X.keys())

    def extract_useful_columns(self, X):
        useful_columns = []
        for i in range(0, X.shape[1]):
            result = np.where(X[:, i] > 0)
            if 5 < len(result[0]) < 500:
                useful_columns.append(i)
        return useful_columns
    
    def transform_y(self, X, y):
        handles = list(y.keys())
        for key in handles:
            if key not in X.keys():
                del y[key]
        return y

    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self

        # Method that describes what we need this transformer to do

    def transform(self, X, y=None):
        if y is not None:
            features = self.mp_deleter(X, y)
        if y is not None:
            y = self.transform_y(features, y)
        return feature_vector, y

In [ ]:
x = joblib.load('./2017vectors100.x')
y = joblib.load('./all_trimmed_mp_2017_brexit_vote_label.y')
real_y = list(y.values())
future_x = joblib.load('./2019vectors100.x')
future_y = joblib.load('./all_new_2019_no_confidence_vote_labels.y')
real_future_y = list(future_y.values())

In [ ]:
transformer = TimeFeatureExtractor()
transformer = transformer.fit(x, y)
x, real_y = transformer.transform(x, y)
future_x, future_y = transformer.transform(future_x, future_y)

In [ ]:
reg = LogisticRegression(C=1)
reg.fit(x, real_y)
predictions = reg.predict(x)
print(classification_report(np.array(list(future_y.values())), predictions, labels=[0,1]))